In [ ]:
%%capture
!pip install torchinfo
import os
import zipfile
import numpy as np
import pandas as pd
import torch
import torch.nn.functional as F
from torch import nn
from torch.utils.data import Dataset, DataLoader, random_split
from torchvision import transforms
from torchinfo import summary
from skimage import io
import matplotlib.pyplot as plt

In [ ]:
DATA_DIR        = '../input/aerial-cactus-identification/'
TRAIN_ZIP_DIR   = DATA_DIR + 'train.zip'
TEST_ZIP_DIR    = DATA_DIR + 'test.zip'
SAMPLE_SUBMIS   = DATA_DIR + 'sample_submission.csv'
ANNOTATIONS_DIR = DATA_DIR + 'train.csv'
TRAIN_DIR       = './train'
TEST_DIR        = './test'

DEVICE          = 'cuda' if torch.cuda.is_available() else 'cpu'
N_LABELS        = 2
N_EPOCHS        = 10
BATCH_SIZE      = 64
LEARNING_RATE   = 0.001
MOMENTUM        = 0.9
LABELS_MAP      = {0: 'No Cactus', 1: 'Cactus'}

def init_weights(layer):
    if type(layer) in [nn.Linear, nn.Conv2d]:
        nn.init.xavier_uniform_(layer.weight)
        layer.bias.data.fill_(0.01)

def display_data(data, n=10, classes=None):
    fig, ax = plt.subplots(1, n, figsize=(15,3))
    indices = np.random.randint(0, len(data), size=n)
    for i, j in enumerate(indices):
        ax[i].imshow(np.transpose(data[j][0], (1, 2, 0)))
        ax[i].axis('off')
        if classes:
            ax[i].set_title(classes[data[j][1]])
            
def train_epoch(model, 
                dataloader, 
                lr=LEARNING_RATE, 
                optimizer=None, 
                loss_fn=nn.NLLLoss()):
    optimizer = optimizer or torch.optim.Adam(model.parameters(), lr=lr)
    model.train()
    total_loss, accuracy, count = 0, 0, 0
    for X, y in dataloader:
        X, y = X.to(DEVICE), y.to(DEVICE)
        optimizer.zero_grad()
        out = model(X)
        loss = loss_fn(out, y)
        loss.backward()
        optimizer.step()
        total_loss += loss
        predicted = torch.max(out, 1)[1]
        accuracy += (predicted == y).sum()
        count += len(y)
    return total_loss.item() / count, accuracy.item() / count

def validate(model, 
             dataloader, 
             loss_fn=nn.NLLLoss()):
    model.eval()
    total_loss, accuracy, count = 0, 0, 0
    with torch.no_grad():
        for X, y in dataloader:
            X, y = X.to(DEVICE), y.to(DEVICE)
            out = model(X)
            total_loss += loss_fn(out, y)
            predicted = torch.max(out, 1)[1]
            accuracy += (predicted == y).sum()
            count += len(y)
    return total_loss.item() / count, accuracy.item() / count 
    
def train(model, 
          train_loader, 
          valid_loader=None, 
          optimizer=None, 
          lr=LEARNING_RATE, 
          epochs=N_EPOCHS, 
          loss_fn=nn.NLLLoss()):
    optimizer = optimizer or torch.optim.Adam(net.parameters(),lr=lr)
    history = {'train_loss': [], 'train_accuracy': []}
    if valid_loader is not None:
        history['validation_loss'] = []
        history['validation_accuracy'] = []
    for epoch in range(epochs):
        tl, ta = train_epoch(model, 
                             train_loader, 
                             lr=lr, 
                             optimizer=optimizer, 
                             loss_fn=loss_fn)
        history['train_loss'].append(tl)
        history['train_accuracy'].append(ta)
        if valid_loader is not None:
            vl, va = validate(model, valid_loader, loss_fn=loss_fn)
            print(f"Epoch {epoch:2}, Train Acc = {ta:.3f}, Val Acc = {va:.3f}, Train Loss = {tl:.3f}, Val Loss={vl:.3f}")
            history['validation_loss'].append(vl)
            history['validation_accuracy'].append(va)
        else:
            print(f"Epoch {epoch:2}, Train Acc = {ta:.3f}, Train Loss = {tl:.3f}")
    return history

def plot_history(history, validation=False):
    plt.figure(figsize=(15, 5))
    plt.subplot(121)
    plt.ylabel('Accuracy')
    plt.xlabel('Epochs')
    plt.plot(history['train_accuracy'], label='Training')
    if validation:
        plt.plot(history['validation_accuracy'], label='Validation')
    plt.legend()
    plt.subplot(122)
    plt.ylabel('Loss')
    plt.xlabel('Epochs')
    plt.plot(history['train_loss'], label='Training')
    if validation:
        plt.plot(history['validation_loss'], label='Validation')
    plt.legend()
    
def submission(dataset, model):
    model.eval()
    result = []
    with torch.no_grad():
        for datapoint in dataset:
            X = datapoint[0][None, ...].to(DEVICE)
            out = model(X)
            result.append([datapoint[1], float(torch.exp(out)[0][1])])
    df = pd.DataFrame(result, columns = ['id', 'has_cactus'])
    df = df.set_index('id')
    df = df.sort_values('id')
    df.to_csv('./submission.csv')
    return df

In [ ]:
if not os.path.exists(TRAIN_DIR):
    with zipfile.ZipFile(TRAIN_ZIP_DIR, 'r') as zip_ref:
        zip_ref.extractall('./')
if not os.path.exists(TEST_DIR):
    with zipfile.ZipFile(TEST_ZIP_DIR, 'r') as zip_ref:
        zip_ref.extractall('./')

In [ ]:
class ACIDataset(Dataset):
    def __init__(self, 
                 img_dir,
                 annotations_file=None, 
                 transform=None, 
                 target_transform=None):
        self.img_dir = img_dir
        self.is_labeled = False
        if annotations_file is not None:
            self.is_labeled = True
            self.img_labels = pd.read_csv(annotations_file)
        else:
            self.img_labels = pd.DataFrame(os.listdir(img_dir))
        self.transform = transform
        self.target_transform = target_transform
        
    def __len__(self):
        return len(self.img_labels)
    
    def __getitem__(self, idx):
        img_path = os.path.join(self.img_dir, 
                                self.img_labels.iloc[idx, 0])
        image = io.imread(img_path)
        if self.transform:
            image = self.transform(image)
        if self.is_labeled:
            label = self.img_labels.iloc[idx, 1]
            if self.target_transform:
                label = self.target_transform(label)
            sample = [image, label]
        else:
            sample = [image, self.img_labels.iloc[idx, 0]]
        return sample

transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))]) 

data = ACIDataset(
    img_dir=TRAIN_DIR,
    annotations_file=ANNOTATIONS_DIR,
    transform=transform,
    target_transform=None)

test_data = ACIDataset(
    img_dir=TEST_DIR,
    transform=transform)

train_data, val_data = random_split(data, [len(data) * 8 // 10, 
                                           len(data) * 2 // 10])

train_dl = DataLoader(train_data, batch_size=BATCH_SIZE)
valid_dl = DataLoader(val_data, batch_size=BATCH_SIZE)
all_dl   = DataLoader(data, batch_size=BATCH_SIZE)
test_dl  = DataLoader(test_data, batch_size=BATCH_SIZE)

In [ ]:
display_data(data, n=12, classes=LABELS_MAP)

In [ ]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(
            in_channels=3,
            out_channels=10,
            kernel_size=(5, 5))
        self.pool = nn.MaxPool2d(
            kernel_size=(2, 2))
        self.conv2 = nn.Conv2d(
            in_channels=10,
            out_channels=20,
            kernel_size=(3, 3))
        self.fc = nn.Linear(
            in_features = 20*6*6,
            out_features = 2)
        
    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(-1, 20*6*6)
        x = F.log_softmax(self.fc(x), dim=1)
        return x

In [ ]:
# model = Net().to(DEVICE)
# model.apply(init_weights)
# summary(model, input_size=(1,3,32,32))

In [ ]:
# optimizer = torch.optim.Adam(model.parameters(), 
#                              lr=LEARNING_RATE)
# history = train(model, 
#                 train_dl, 
#                 valid_dl, 
#                 optimizer=optimizer, 
#                 lr=LEARNING_RATE, 
#                 epochs=N_EPOCHS, 
#                 loss_fn=nn.NLLLoss())

In [ ]:
# plot_history(history, validation=True)

In [ ]:
%%capture
model_ = Net().to(DEVICE)
model_.apply(init_weights)

In [ ]:
optimizer = torch.optim.Adam(model_.parameters(), 
                             lr=LEARNING_RATE)
history = train(model_, 
                all_dl, 
                optimizer=optimizer, 
                lr=LEARNING_RATE, 
                epochs=N_EPOCHS, 
                loss_fn=nn.NLLLoss())

In [ ]:
plot_history(history)

In [ ]:
submission(test_data, model_)